In [1]:
import simpy

In [2]:
'''
Car is a process
After parking, it will yield the tiemout event and signal the simulation that the process wants
to wait for that event to occur
Once the event occurs the process will resume the operation

It takes the env as the input to indicate that car is the process in this env
'''
def car (env):
    while (True):
        print("Car starting at %d" % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [3]:
env = simpy.Environment()
env.process(car(env))                #making the car as the process 
                                     #creating the instance of the process
env.run(until = 15)                  #running the envirionment for 15 timesteps

Car starting at 0
Start driving at 5
Car starting at 7
Start driving at 12
Car starting at 14


In [4]:
#creating two differenct processes and then calling it
def proc1 (env):
    while (True):
        print ("Process 1 is invoked at {}".format(env.now))
        yield env.timeout(2)

def proc2 (env):
    while (True):
        print ("Process 2 is invoked at {}".format(env.now))
        yield env.timeout(3)

In [5]:
env = simpy.Environment()
env.process (proc1(env))
env.process (proc2(env))
env.run(until= 15)

Process 1 is invoked at 0
Process 2 is invoked at 0
Process 1 is invoked at 2
Process 2 is invoked at 3
Process 1 is invoked at 4
Process 2 is invoked at 6
Process 1 is invoked at 6
Process 1 is invoked at 8
Process 2 is invoked at 9
Process 1 is invoked at 10
Process 2 is invoked at 12
Process 1 is invoked at 12
Process 1 is invoked at 14


# Process Interaction

In [6]:
'''
Car object is created 
There are two processes in the car class : run , charge

When the car object is created then the run process is automatically created
A new charge process is started every time the vehicle starts parking. 
By yielding the Process instance that Environment.process() returns, the run process starts waiting for it to finish

'''
class Car(object):
    def __init__(self, env):
        #Car object have the env instance in which it is instantiated
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            try : 
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print("Was interrupted. Hope , the battery is full enough ...")
            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    def charge(self, duration):
        yield self.env.timeout(duration)

In [7]:
def driver (env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [9]:
#running the instance
env = simpy.Environment()
car = Car(env)
env.process(driver(env , car))
env.run(until = 20)

Start parking and charging at 0
Was interrupted. Hope , the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12
Start driving at 17
Start parking and charging at 19


# Resource Usage

In [10]:
def car_resource(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)
    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        # Charge the battery
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))

In [11]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity = 2 )

In [12]:
#create 4 processes car_resource
#All of them drive first and then try to acquire the resource
for i in range (4):
    env.process(car_resource(env, 'Car %d' %i , bcs , i * 2 , 5))

In [13]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12
